# Objectif

Ce projet a pour but de développer un modèle de classification binaire capable de prédire la polarité (positive ou négative) d'une critique de film issue du site IMDb, en utilisant exclusivement son contenu textuel.

Il s’inscrit dans le cadre de l’apprentissage automatique supervisé, et utilise un réseau de neurones à mémoire longue courte durée (LSTM) construit avec TensorFlow/Keras.

# Données 

Les données proviennent du corpus IMDb Large Movie Review Dataset disponible ici :
🔗 https://ai.stanford.edu/~amaas/data/sentiment/

50 000 critiques au total, réparties en :

train/pos : 12 500 critiques positives

train/neg : 12 500 critiques négatives

test/pos : 12 500 critiques positives

test/neg : 12 500 critiques négatives

Chaque critique est un fichier .txt contenant le texte d’une critique

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam

1. Chargement des données IMDb

In [ ]:
def load_imdb_data(data_dir):
    texts, labels = [], []
    for label_type in ['neg', 'pos']:
        dir_name = os.path.join(data_dir, label_type)
        for fname in os.listdir(dir_name):
            if fname.endswith(".txt"):
                with open(os.path.join(dir_name, fname), encoding="utf-8") as f:
                    texts.append(f.read())
                labels.append(0 if label_type == 'neg' else 1)
    return texts, labels

2. Prétraitement

In [ ]:
train_dir = "data/aclImdb/train"
test_dir = "data/aclImdb/test"

x_train_texts, y_train = load_imdb_data(train_dir)
x_test_texts, y_test = load_imdb_data(test_dir)